### Homework Assignment 1: Analyzing Sales Data

In [1]:
import pandas as pd 
import numpy as np 

In [2]:
df=pd.read_csv(r"C:\Users\Musaffa\Downloads\Telegram Desktop\sales_data.csv")
df.head()

,Date,Product,Category,Quantity,Price
0,2023-01-01,Laptop,Electronics,10,800
1,2023-01-01,T-Shirt,Clothing,5,20
2,2023-01-02,Smartphone,Electronics,8,400
3,2023-01-02,Coffee Maker,Home,12,50
4,2023-01-03,Jeans,Clothing,15,30


In [4]:
#1. 

result=df.groupby('Category').agg({'Quantity': 'sum'})
print(result)

             Quantity
Category             
Clothing          157
Electronics       183
Home              144


In [5]:
Avg_price=df.groupby('Category').agg({'Price': 'mean'})
print(Avg_price)

                  Price
Category               
Clothing      31.176471
Electronics  276.764706
Home          55.000000


In [ ]:
max_quant=df.groupby('Category').agg({'Quantity': 'max'})
print(max_quant)

             Quantity
Category             
Clothing           15
Electronics        15
Home               14


In [19]:
#2. Identify the top-selling product in each category based on the total quantity sold.

result=df.groupby(['Category', 'Product']).agg({'Quantity' : 'sum'})

top_products=result.sort_values('Quantity', ascending=False).groupby('Category'). head(1)
print(top_products)

                             Quantity
Category    Product                  
Clothing    Jeans                  15
Electronics Soundbar               15
Home        Pressure Cooker        14


In [28]:
#3. Find the date on which the highest total sales (quantity * price) occurred.

df['Date']= pd.to_datetime(df['Date'])
df['Sales']=df['Quantity'] * df['Price']

result=df.groupby('Date').agg({'Sales' : 'max'})
max_sales= result.sort_values('Sales', ascending=False).groupby('Date').head(1)
print(max_sales.head(1))


            Sales
Date             
2023-01-07  15000


### Homework Assignment 2: Examining Customer Orders

In [30]:
df=pd.read_csv(r"C:\Users\Musaffa\Downloads\Telegram Desktop\customer_orders.csv")
df.head()

,OrderID,CustomerID,Product,Quantity,Price
0,1,101,Laptop,2,800
1,2,102,Headphones,1,150
2,3,103,Smartphone,3,400
3,4,101,External Hard Drive,2,80
4,5,102,Backpack,1,40


In [ ]:
result=df.groupby('CustomerID')['Quantity'].count()
under_20= result[result < 20].index
filtered_df= df[df['CustomerID']. isin(under_20)]
print(filtered_df)

    OrderID  CustomerID                 Product  Quantity  Price
0         1         101                  Laptop         2    800
1         2         102              Headphones         1    150
3         4         101     External Hard Drive         2     80
4         5         102                Backpack         1     40
6         7         101          Wireless Mouse         1     25
8         9         102                Smart TV         1   1000
13       14         101              Sunglasses         1     15
14       15         102                 Blender         2     60
18       19         101          Gaming Console         1    400
19       20         102            Summer Dress         2     30
23       24         101             Cargo Pants         2     30
24       25         102               Desk Lamp         1     25
28       29         101            Denim Jacket         1     45
29       30         102            Casual Shoes         1     30
33       34         101  

In [45]:
#2. Identify customers who have ordered products with an average price per unit greater than $120.

result=df.groupby('CustomerID').agg({'Price': 'mean'})
print(result)
filtered_df=result[result>120]
print(filtered_df)

                 Price
CustomerID            
101         110.714286
102         138.095238
103          72.000000
104         169.750000
105          85.277778
                 Price
CustomerID            
101                NaN
102         138.095238
103                NaN
104         169.750000
105                NaN


In [49]:
#3. Find the total quantity and total price for each product ordered, and filter out products that have a total quantity less than 5 units.

result=df.groupby('Product'). agg({'Quantity' : 'sum', 'Price' :'sum'})
print(result.head())

filtered_df=result[result['Quantity'] < 5]
print(filtered_df)

                   Quantity  Price
Product                           
Backpack                  4    160
Blender                   4    180
Blu-ray Player            2    140
Bluetooth Headset         2    100
Bookshelf                 2    160
                        Quantity  Price
Product                                
Backpack                       4    160
Blender                        4    180
Blu-ray Player                 2    140
Bluetooth Headset              2    100
Bookshelf                      2    160
Casual Shoes                   3     90
Coffee Maker                   3    150
Coffee Table                   3    200
Comforter Set                  3    135
Cookware Set                   1     60
Denim Jacket                   2     90
Denim Skirt                    1     25
Desk Chair                     2    150
Desk Lamp                      2     50
Drone                          2    600
External Hard Drive            3    160
Fitness Tracker                3   

In [1]:
import sqlite3
import pandas as pd 

#### Homework Assignment 3: Population Salary Analysis

In [3]:
conn = sqlite3.connect("population.db")
population_df = pd.read_sql_query("SELECT * FROM population", conn)

In [5]:
salary_bands = pd.DataFrame({
    'Salary Band': ['Low', 'Medium', 'High', 'Very High'],
    'Min': [0, 50000, 100000, 150000],
    'Max': [49999, 99999, 149999, float('inf')]
})


In [6]:
def classify_salary(salary):
    for _, row in salary_bands.iterrows():
        if row['Min'] <= salary <= row['Max']:
            return row['Salary Band']
    return 'Unknown'

In [7]:
population_df['Salary Band'] = population_df['salary'].apply(classify_salary)

In [8]:
state_summary = population_df.groupby(['state', 'Salary Band']).agg(
    Population=('salary', 'count'),
    Average_Salary=('salary', 'mean'),
    Median_Salary=('salary', 'median')
).reset_index()

In [9]:
state_totals = population_df.groupby('state')['salary'].count().rename('TotalStatePop').reset_index()
state_summary = state_summary.merge(state_totals, on='state')
state_summary['Percentage'] = (state_summary['Population'] / state_summary['TotalStatePop']) * 100
state_summary.drop(columns='TotalStatePop', inplace=True)

In [10]:
state_summary.to_excel("state_salary_band_analysis.xlsx", index=False)

In [11]:
print(state_summary.head(10))

     state Salary Band  Population  Average_Salary  Median_Salary  Percentage
0  Alabama        High          10    1.264651e+05       122833.0    4.065041
1  Alabama         Low           2    2.580350e+04        25803.5    0.813008
2  Alabama      Medium           3    7.713900e+04        78730.0    1.219512
3  Alabama   Very High         231    1.031691e+06      1053186.0   93.902439
4   Alaska        High           2    1.277575e+05       127757.5    3.703704
5   Alaska         Low           2    4.878500e+03         4878.5    3.703704
6   Alaska   Very High          50    1.079418e+06      1062679.5   92.592593
7  Arizona        High           6    1.270102e+05       130715.0    2.371542
8  Arizona         Low           6    2.029317e+04        23208.0    2.371542
9  Arizona      Medium          10    7.577330e+04        70621.5    3.952569
